# Dislocation Quadrupoles
Dislocation Quadrupoles are structures designed with the goal of enabling the study of dislocation systems in periodic cells. In order for the system to be periodic, the cell must have a net Burgers vector of zero. To achieve this, we place two dislocations with Burgers vectors of equal magnitude and opposite sign (i.e. $+\textbf{b}$ and $-\textbf{b}$) in the structure.

The code is designed to allow the generation of these structures in as small a structure is required for a given separation of the dislocations along the glide direction.

To start, lets look at the $90^\circ$ partial dislocation in Diamond, using a dislocation cylinder approach documented in earlier docs and the same potential by [D. Holland and M. Marder](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.80.746):

In [ ]:
from matscipy.dislocation import DiamondGlide90degreePartial, get_elastic_constants
from matscipy.calculators.manybody.explicit_forms.stillinger_weber import StillingerWeber,\
                                                                Holland_Marder_PRL_80_746_Si
from matscipy.calculators.manybody import Manybody
calc = Manybody(**StillingerWeber(Holland_Marder_PRL_80_746_Si))

alat, C11, C12, C44 = get_elastic_constants(calculator=calc, symbol="Si", verbose=False)


Si_disloc = DiamondGlide90degreePartial(alat, C11, C12, C44, symbol="Si")

bulk, disloc = Si_disloc.build_cylinder(radius=20)

Si_disloc.view_cyl(disloc, scale=0.3, add_bonds=True)


In [ ]:
from matscipy.dislocation import CubicCrystalDislocationQuadrupole

quad = CubicCrystalDislocationQuadrupole(DiamondGlide90degreePartial, alat, C11, C12, C44, symbol="Si")

quad_bulk, quad_disloc = quad.build_quadrupole(glide_separation=4, self_consistent=False)

view = quad.view_quad(quad_disloc, scale=0.3, add_bonds=True)
view.control.zoom(0.2)
view

## Dislocation Glide in quadrupoles
As quadrupoles are extremely useful for modelling dislocations using plane-wave DFT, it can be convenient to be able to set up initial guesses for complex processes such as dislocation glide. In this scenario, we assume that the full infinite dislocation line glides in unison, ignoring the true "kink" process.

We can use the function `build_glide_quadrupoles` to construct a set of images for this system, which can optionally model the glide of either the "left" ($+\mathbf{b}$) or "right" ($-\mathbf{b}$) dislocation cores, or both at once.

In [ ]:
num_images = 4

glide_quads = quad.build_glide_quadrupoles(nims=num_images, 
                                            glide_left=True, # Let the left dislocation glide
                                            glide_right=True, # Let the left dislocation glide
                                            self_consistent=False,
                                            glide_separation=4,
                                            verbose=False)

In [ ]:
# image 0 will be identical to the above structure
quad.view_quad(glide_quads[1], scale=0.3, add_bonds=True)

In [ ]:
quad.view_quad(glide_quads[2], scale=0.3, add_bonds=True)

In [ ]:
quad.view_quad(glide_quads[3], scale=0.3, add_bonds=True)